# Solution analytique en tube

In [ ]:
import copy
import os, sys
import time
import math

# See Marfaing, O., Guingo, M., Laviéville, J., Bois, G., Méchitoua, N., Mérigoux, N., & Mimouni, S. (2016). An analytical relation for the void fraction distribution in a fully developed bubbly flow in a vertical pipe. Chemical Engineering Science, 152, 579-585.

In [ ]:
from trustutils import run

run.TRUST_parameters()

In [ ]:
sondes = {"Pipe":{}, "Channel2D":{}, "Channel3D":{}}


n_seg_sonde = 40
r_tube = 0.019

tab_name_sondes = ["vitesse_liquide_eau",
                   "temperature_liquide_eau"
                  ]

x_0 = r_tube/(2*n_seg_sonde)*math.cos(math.pi*.5/360)
y_0 = r_tube/(2*n_seg_sonde)*math.sin(math.pi*.5/360)

x_wall = r_tube*(2*n_seg_sonde-1)/(2*n_seg_sonde)*math.cos(math.pi*.5/360)
y_wall = r_tube*(2*n_seg_sonde-1)/(2*n_seg_sonde)*math.sin(math.pi*.5/360)

z_sonde = 1.49

str_sonde = " "
for name_sonde in tab_name_sondes:
    str_sonde += f" {name_sonde} {name_sonde} periode 1.e-2 segment {n_seg_sonde} {x_0} {y_0} {z_sonde} {x_wall} {y_wall} {z_sonde}  \n"

sondes["Pipe"]["PolyMAC"],sondes["Channel3D"]["PolyMAC"],sondes["Channel3D"]["VDF"]  = str_sonde, str_sonde, str_sonde
sondes["Pipe"]["PolyVEF"],sondes["Channel3D"]["PolyVEF"]  = str_sonde, str_sonde

str_sonde = " "
for name_sonde in tab_name_sondes:
    str_sonde += f" {name_sonde} {name_sonde} periode 1.e-2 segment {n_seg_sonde} {r_tube/(2*n_seg_sonde)} {z_sonde} {r_tube*(2*n_seg_sonde-1)/(2*n_seg_sonde)} {z_sonde}  \n"

sondes["Pipe"]["VDF"],sondes["Channel2D"]["PolyMAC"],sondes["Channel2D"]["VDF"]  = str_sonde, str_sonde, str_sonde
sondes["Channel2D"]["PolyVEF"] = str_sonde


In [ ]:
#Create the calculation cases

force_recalculation = False 

# Refinement
geometry   = [ "Pipe", "Channel2D", "Channel3D", ]
med_file   = {  "VDF" : "",  "PolyMAC" : "1_tube_analytique.med", "PolyVEF" : "1_tube_analytique.med"  }
name_mesh  = { "Pipe" : { "PolyMAC" : "read_med { domain dom mesh mesh file 1_tube_analytique.med }" ,
                          "PolyVEF" : "read_med { domain dom mesh mesh file 1_tube_analytique.med }" ,
                          "VDF" : "Mailler dom { Pave tube  { Origine 0. 0. Nombre_de_Noeuds 41 201 Longueurs  0.019 1.5 } { bord symetrie    X = 0   0 <= Y <= 1.5  bord top   Y = 1.5   0 <= X <= 0.019  bord bottom    Y = 0   0 <= X <= 0.019 bord wall   X = 0.019 0 <= Y <= 1.5 } } "},
               "Channel2D" : { "PolyMAC" : "Mailler dom { Pave tube  { Origine 0. 0. Nombre_de_Noeuds 41 201 Longueurs  0.019 1.5 } { bord symetrie    X = 0   0 <= Y <= 1.5  bord top   Y = 1.5   0 <= X <= 0.019  bord bottom    Y = 0   0 <= X <= 0.019 bord wall   X = 0.019 0 <= Y <= 1.5 } } " ,
                               "PolyVEF" : "Mailler dom { Pave tube  { Origine 0. 0. Nombre_de_Noeuds 41 201 Longueurs  0.019 1.5 } { bord symetrie    X = 0   0 <= Y <= 1.5  bord top   Y = 1.5   0 <= X <= 0.019  bord bottom    Y = 0   0 <= X <= 0.019 bord wall   X = 0.019 0 <= Y <= 1.5 } } " ,
                               "VDF"     : "Mailler dom { Pave tube  { Origine 0. 0. Nombre_de_Noeuds 41 201 Longueurs  0.019 1.5 } { bord symetrie    X = 0   0 <= Y <= 1.5  bord top   Y = 1.5   0 <= X <= 0.019  bord bottom    Y = 0   0 <= X <= 0.019 bord wall   X = 0.019 0 <= Y <= 1.5 } } "},
               "Channel3D" : { "PolyMAC" : "Mailler dom { Pave tube  { Origine 0. 0. 0. Nombre_de_Noeuds 41 2 201  Longueurs  0.019 0.1 1.5  } { bord symetrie    X = 0   0 <= Y <= 0.1 0 <= Z <= 1.5  bord symetrie   Y = 0. 0 <= X <= 0.019   0 <= Z <= 1.5   bord symetrie  Y = 0.1 0 <= X <= 0.019   0 <= Z <= 1.5   bord top   Z = 1.5   0 <= X <= 0.019  0 <= Y <= 0.1 bord bottom    Z = 0   0 <= X <= 0.019  0 <= Y <= 0.1 bord wall   X = 0.019  0 <= Y <= 0.1 0 <= Z <= 1.5  } } polyedriser dom " ,
                               "PolyVEF" : "Mailler dom { Pave tube  { Origine 0. 0. 0. Nombre_de_Noeuds 41 2 201  Longueurs  0.019 0.1 1.5  } { bord symetrie    X = 0   0 <= Y <= 0.1 0 <= Z <= 1.5  bord symetrie   Y = 0. 0 <= X <= 0.019   0 <= Z <= 1.5   bord symetrie  Y = 0.1 0 <= X <= 0.019   0 <= Z <= 1.5   bord top   Z = 1.5   0 <= X <= 0.019  0 <= Y <= 0.1 bord bottom    Z = 0   0 <= X <= 0.019  0 <= Y <= 0.1 bord wall   X = 0.019  0 <= Y <= 0.1 0 <= Z <= 1.5  } } polyedriser dom " ,
                               "VDF" :     "Mailler dom { Pave tube  { Origine 0. 0. 0. Nombre_de_Noeuds 41 2 201  Longueurs  0.019 0.1 1.5  } { bord symetrie    X = 0   0 <= Y <= 0.1 0 <= Z <= 1.5  bord symetrie   Y = 0. 0 <= X <= 0.019   0 <= Z <= 1.5   bord symetrie  Y = 0.1 0 <= X <= 0.019   0 <= Z <= 1.5   bord top   Z = 1.5   0 <= X <= 0.019  0 <= Y <= 0.1 bord bottom    Z = 0   0 <= X <= 0.019  0 <= Y <= 0.1 bord wall   X = 0.019  0 <= Y <= 0.1 0 <= Z <= 1.5  } }  "}
             }         
#no VDF for now : problem in velocity derivative
schema = { "VDF" : " vdf  dis option_vdf { all_options } ", 
           "PolyMAC" : " PolyMAC_P0  dis option_PolyMAC_P0 { traitement_axi } ", 
           "PolyVEF" : " PolyVEF_P0  dis option_PolyVEF_P0 { traitement_axi } "  }
#schema = { "PolyMAC" : " PolyMAC_P0  dis option_PolyMAC_P0 { traitement_axi } " }
dimension = { "Pipe"      :{ "VDF" : 2, "PolyMAC" : 3, "PolyVEF" : 3}, 
              "Channel2D" :{ "VDF" : 2, "PolyMAC" : 2, "PolyVEF" : 2}, 
              "Channel3D" :{ "VDF" : 3, "PolyMAC" : 3, "PolyVEF" : 3} }
bidim_axi = { "Pipe" :{ "VDF" : "bidim_axi",  "PolyMAC" : " ",  "PolyVEF" : " "}, 
              "Channel2D" :{ "VDF" : " ",  "PolyMAC" : " ",  "PolyVEF" : " "}, 
              "Channel3D" :{ "VDF" : " ",  "PolyMAC" : " ",  "PolyVEF" : " "} }
nrad = { "VDF" : 40,  "PolyMAC" : 40}
solveur = { "ICE" : "", "SETS" : " 	facsec_diffusion_for_sets 100 " }
#solveur = { "SETS" : " 	facsec_diffusion_for_sets 100 " } # Only SETS for fast run on TRUST validation
paroi = { "VDF" : " symetrie ",  "PolyMAC" : " paroi ",  "PolyVEF" : " paroi "}
paroi_adiabatique = { "VDF" : " symetrie ", "PolyMAC" : " paroi_adiabatique ", "PolyVEF" : " paroi_adiabatique "}

name_geom = {}
name_conf = {}

# Numerical parameters
facsec  = 1
tmax    = 3
nb_pas_dt_max = '100000'
seuil_statio  = '1.e-3'
number_of_partitions = {"Channel2D_PolyMAC_SETS" : 1, "Channel3D_PolyMAC_SETS" : 2, "Pipe_PolyMAC_SETS":8,
                        "Channel2D_PolyVEF_SETS" : 1, "Channel3D_PolyVEF_SETS" : 2, "Pipe_PolyVEF_SETS":8,
                        "Channel2D_VDF_SETS" : 1, "Channel3D_VDF_SETS" : 1, "Pipe_VDF_SETS": 1}
number_of_partitions = {"Channel2D_PolyMAC_SETS" : 4, "Channel2D_PolyMAC_ICE" : 4,
                        "Channel2D_VDF_SETS" : 4, "Channel2D_VDF_ICE" : 4}

#Physical quantities
u_0 = 1

#Turbulent quantities
D_h = 0.038
CI_om = 10*u_0/D_h
CI_k  = 0.01*u_0**2

build_dir = run.BUILD_DIRECTORY
#build_dir = "/volatile/catB/cr268029/trust/Validation/Rapports_automatiques/Verification/Multiphase/Tube_solution_analytique/build"

for g in geometry:
    for s in schema.keys() :
        for solv in solveur.keys() :
            name = f"{g}_{s}_{solv}"
            if name in number_of_partitions.keys() :
                name_geom[name] = g
                name_conf[name] = f"{s}_{solv}"


if force_recalculation or not os.path.exists(f'{build_dir}'):
    print('Recalculation running..')
    run.reset()
    for g in geometry:
        for s in schema.keys() :
            for solv in solveur.keys() :
                name = f"{g}_{s}_{solv}"
                if name in number_of_partitions.keys() :

                    substitutions_dict = {"dimension" : dimension[g][s],
                                            "bidim_axi" : bidim_axi[g][s],
                                            "name_mesh" : f'{name_mesh[g][s]}' ,
                                            "schema" : schema[s],

                                            "tmax": str(tmax),                                  
                                            "facsec" : str(facsec) ,
                                            "nb_pas_dt_max" : nb_pas_dt_max ,
                                            "seuil_statio" : seuil_statio ,
                                            "solveur" : solv ,
                                            "facsec_diffusion_for_sets" : solveur[solv] ,

                                            "u_0"  : f"{1*dimension[g][s]} {''.join([ ' 0 ' for i in range(1*(dimension[g][s]-1))])} {u_0} " ,

                                            "paroi_adiabatique" : paroi_adiabatique[s],
                                            "paroi" : paroi[s],

                                            "sondes" : sondes[g][s],

                                        }

                    tc = run.addCaseFromTemplate("jdd.data",targetDirectory=f"{name}",dic=substitutions_dict,nbProcs=number_of_partitions[name])
                    if (med_file[s]!=""):
                        os.system(f'cp {build_dir}/{med_file[s]} {build_dir}/{name}/{med_file[s]}')

                    if number_of_partitions[name] > 1:
                        tc.partition()
    run.printCases()
    run.runCases()
    display(run.tablePerf())

else:
    print('No recalculation, reusing old results!')

In [ ]:
from IPython.display import display
from trustutils import visit
from trustutils import plot
import numpy as np
import pandas as pd

In [ ]:
## Loading simulation results

############# results : sim[config][debit air][raf][position]
#############         columns : ['y/h'] ['vxl'] ['vyl'] ['vxg'] ['vyg'] ['k'] ['alp'] ['DB']
#############                   ["diff"] ["conv"] ["press"] ["grav"] ["drag"] ["lift"] ["disp"] ["lub"]
#############               nb : end column name with lx for forces
############# sim name : tab_name [config][debit air][raf]
############# sim label: tab_label[config][debit air][raf]
############# convergence :  dt_ev[config][debit air][raf]
#############         columns : ['time']['dt']['facsec']['residu']['dt_stab']['vitl']['vitg']['alphal']['alphag']['Tl']['Tg']['diss']['k'] 

sim      =  {  }
dt_ev    = {  }

for s in schema.keys() :
    for g in geometry:
        for solv in solveur.keys() :
            name = f"{g}_{s}_{solv}"
            if name in number_of_partitions.keys() :

                par = ""
                if number_of_partitions[name] > 1 : par = "PAR_"

                dt_ev[name] = plot.read_csv(f"{name}/{par}jdd.dt_ev", sep="\t")
                dt_ev[name] = dt_ev[name].iloc[:,:8]
                dt_ev[name].columns = ['time', 'dt', 'facsec', 'residu', 'dt_stab', 'vitl', 'alphal', 'tl']               

                sim[name] = pd.DataFrame()
                sim[name]['r+']= np.linspace(1./(2*n_seg_sonde),1-1./(2*n_seg_sonde), 40)
                sim[name]['vxl']=np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_LIQUIDE_EAU.son"))[1::dimension[g][s],-1]
                sim[name]['vzl']=np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_LIQUIDE_EAU.son"))[dimension[g][s]::dimension[g][s],-1]
                sim[name]['tl']= np.array(plot.loadText(f"{name}/{par}jdd_TEMPERATURE_LIQUIDE_EAU.son"))[1::,-1]


In [ ]:
## plotting convergence
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['text.usetex'] = True

matplotlib.rcParams.update({'font.size': 14})

color_conf = { "VDF_ICE" : "red", "PolyMAC_ICE" : "magenta", "VDF_SETS" : "orange", "PolyMAC_SETS" : "purple"  }
marker_geom =  { "Pipe" : "-", "Channel2D" : "--", "Channel3D" : ":"}
linesize_geom =  { "Pipe" : 1, "Channel2D" : 2, "Channel3D" : 4}

fig = plt.figure(figsize = (10, 5))

name_phys = [ ["vitl"], ["tl"]]

axs = fig.subplots(1, len(name_phys) )#, sharex=True, sharey=True)

for l in range(len(name_phys)): #phys
    for r in dt_ev.keys() :
        for lp in range(len(name_phys[l])) : #physique de la phase si multiphase
            axs[l].plot(dt_ev[r]["time"][::], dt_ev[r][name_phys[l][lp]][::], marker_geom[name_geom[r]], label = r.replace("_", " "), color = color_conf[name_conf[r]], linewidth = linesize_geom[name_geom[r]])

        axs[l].set_xlim(0,tmax)
        axs[l].set_ylim(float(seuil_statio)/100, 1e4)
        axs[l].set_yscale("log")
        axs[l].set_title("convergence "+name_phys[l][0] )
        axs[l].set_xlabel('time')
        axs[l].set_ylabel("convergence " + name_phys[l][0])
        axs[l].legend(prop={'size': 10})

plt.tight_layout()
#plt.savefig(f"{build_dir}/convergence.pdf")

In [ ]:
# Afficher tous les résultats de simu

# Colonnes : Titre, alpha, d, U (contient U_l et U_g), u'l
    
color_anap = "cyan"
color_anac = "blue"
marker_ana= "-"

plt.figure(figsize = (12,6))

plt.subplot(121)

plt.xlim(0,1)
plt.ylim(0,1.4)
for s in dt_ev.keys():
    par = ""
    if number_of_partitions[s] > 1 : par = "PAR_"

    f = os.open(f"{build_dir}/{s}/{par}jdd.perf", os.O_RDONLY)
    time_loc = str(float(os.read(f, 200).split()[3])/60*number_of_partitions[s])[:5]
    plt.plot(np.array(sim[s]["r+"]), sim[s]['vzl'], marker_geom[name_geom[s]], label = f"TrioCFD {s.replace('_',' ')} {nrad[s.split('_')[1]]} rad elems {time_loc}min*proc calc", color = color_conf[name_conf[s]], linewidth = linesize_geom[name_geom[s]])

plt.legend(fontsize = 8)
plt.xlabel(r"$r_+$")
plt.ylabel("Velocity")
plt.title("Comparison between Analytical and TRUST results")
    
plt.subplot(122)

plt.xlim(0,1)
plt.ylim(0,1.4)
for s in dt_ev.keys():
    plt.plot(np.array(sim[s]["r+"]), sim[s]['tl'], marker_geom[name_geom[s]], label = f"TrioCFD {s.replace('_',' ')} {nrad[s.split('_')[1]]} rad elems", color = color_conf[name_conf[s]], linewidth = linesize_geom[name_geom[s]])

plt.legend(fontsize = 8)
plt.xlabel(r"$r_+$")
plt.ylabel("Temperature")
plt.title("Zoom of the near-wall region")

plt.tight_layout()

#plt.savefig(f"{build_dir}/Resultats_tube_analytique.pdf")
